In [ ]:
from numba import jit
import numpy as np, pandas as pd, datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis, Hot
from datashader.utils import export_image
from colorcet import palette
palette["viridis"]=viridis
palette["inferno"]=inferno
palette["Hot"]=Hot


In [ ]:
"""@jit(nopython=True)
def prog08a(cadena, npoints):
    def tr(i): return (i-32)*9.5/95-4.5
    a = [ord(cadena[0])-66]
    for c in cadena[1:]:
        a.append(tr(ord(c)))
    
    x,y = .1,.1
    g=int(a[0])
    gm = g*(g+1)//2
    xs=[0]
    ys=[0]
    for n in range(npoints):
        index=1
        xn,yn = 0,0
        for i in range(g):
            for j in range(g-i):
                xn += a[index]*(x**i)*(y**j)
                yn += a[index+gm]*(x**i)*(y**j)
                index+=1
        xs.append(xn)
        ys.append(yn)
        x, y = xn, yn
    return xs, ys"""

#Sprott's basic translation table (one decimal for Python's round error)
@jit(nopython=True)
def tr(i): return round((i - 32) * 9.5 / 95 - 4.5, 1)

# 2-D, g-degree general polynomials
@jit(nopython=True)
def prog08aa(param_str, npoints):
    a = [ord(param_str[0]) - 66]
    for c in param_str[1:]:
        a.append(tr(ord(c)))
    x, y = .05, .05  # initial conditions (very important!)
    g = int(a[0])  # 'E' quadratic, 'F' cubic, etc.
    gm = g * (g + 1) // 2  # num of param for each variable
    xs, ys = [], []  # store coords
    for n in range(npoints):
        index = 1  # index=0 stores degree
        xn, yn = 0, 0
        # see an example of Sprott's parameters order at Eq 3F pag 80
        for i in range(g):
            xn += a[index] * (x ** i)
            yn += a[index + gm] * (x ** i)
            index += 1
        for i in range(g-2, -1, -1):
            for j in range(1, g-i):
                xn += a[index] * (x ** i) * (y ** j)
                yn += a[index + gm] * (x ** i) * (y ** j)
                index += 1
        xs.append(xn)
        ys.append(yn)
        x, y = xn, yn
    return xs, ys

"""@jit(nopython=True)
def prog08b(param_str, npoints):
    def tr(i): return round((i-32)*9.5/95-4.5,1)
    a = [ord(param_str[0])-66]
    for c in param_str[1:]:
        a.append(tr(ord(c)))
    
    g=int(a[0])
    x,y,z = [.1]*3
    ga = g*(g+1)//3
    gb = g*(g+1)*2//3
    #print(g,ga,gb)
    xs,ys,zs=[],[],[]
    
    for n in range(npoints):
        index=1
        xn,yn,zn = 0,0,0
        for i in range(g):
            for jk in range(g-i):
                for j in range(jk+1):
                    k=jk-j
                    xn += a[index]*(x**k)*(y**j)*(z**i)
                    yn += a[index+ga]*(x**k)*(y**j)*(z**i)
                    zn += a[index+gb]*(x**k)*(y**j)*(z**i)
                    index+=1
        xs.append(xn)
        ys.append(yn)
        zs.append(zn)
        x, y, z = xn, yn, zn
    return xs, ys, zs"""

In [ ]:
def plot(param_strs, prog=prog08aa, npoints=10000000, columns=1, prefix=None, save_path=None):
    cv = ds.Canvas(plot_width = 2400, plot_height = 2400)
    rs = []
    fmt = ".png"
    for idx, param_str in enumerate(param_strs):
        xs, ys = prog(param_str, npoints)
        df = pd.DataFrame({'x':xs, 'y':ys})
        agg = cv.points(df, 'x', 'y')
        r = tf.shade(agg, cmap=palette["Hot"])
        r = tf.set_background(r, 'black')
        rs.append(r)
        if save_path is not None:  # creates path if not already there
            from string import ascii_lowercase
            if prefix is None:
                filename = param_str # prefix = None - param_str
            else:
                filename = prefix + ascii_lowercase[idx] 
            export_image(img=r, filename=filename, fmt=fmt,  export_path=save_path)
            print(f"Saving {save_path}{filename}{fmt}")
    return tf.Images(*rs).cols(columns)

In [ ]:
Fig3_2ss = ["EAGHNFODVNJCP", "EBCQAFMFVPXKQ", "EDSYUECINGQNV", "EELXAPXMPQOBT", "EEYYMKTUMXUVC",
            "EJTTSMBOGLLQF", "ENNMJRCTVVTYG", "EOUGFJKDHSAJU", "EQKOCSIDVTPGY", "EQLOIARXYGHAJ",
            "ETJUBWEDNRORR", "ETSILUNDQSIFA", "EUEBJLCDISIIQ", "EVDUOTLRBKTJD", "EWLKWPSMOGIGS",
            "EZPMSGCNFRENG",
           ]
plot(Fig3_2ss, prefix="Numerical_experiments_fig06", save_path='./') # by defining save_path figures are saved
print("fin")  #comment out this line to view them on screen

In [ ]:
Fig3_18ssa = ["FIRPGUTFIDGCSXMFPKIDJ",
             "FISMHQCHPDFKFBKEALIFD",
             "FJYCBMNFNYOEPYUGHHESU",
             "FLGROKJFELDGKXSUEEWYE",
             "FMGGNDPHWONKFQUIIHBVP",
             "FNHZBEETDORVLAOTUPENH",
             "FNUYLCURDUHQUQMRZQWQB",
             "FOVFKWKEIBPGNYPVKWCYU"]
Fig3_18ssb = ["GFUXRRRUIRDYKDUBPHHOMOBRIRBINCS",
             "GGNXVYVASWMMNFFQOFJTMRBNRFWREJH",
             "GLURFSRHWMSKHTQBKXJDXQSMFJBWUFG",
             "GPFMQPPBPARCUOLSTATEXQDKEXMLOIF",
             "GQDIDSBTPNDBSGOKOKGAKMCCONXFHWQ",
             "GRMJQBCSOAFMBRRSSUHCNBWVSRICXAA",
             "GTPMJKFSCWUMSHBVPCBUTBRRUXHSXIT",
             "GUETJGIINOTHGFYLJOUVEEMXTEGDHLM"]
Fig3_18ssc = ["HGEQGOYIKQQPEUJBKPXTVUSJHOVJDUAYYPRNTXFLGAM",
             "HHVOIEGIDJCSFUFJCQGRUGMCLHEPWKRCCYFIRQPYAPH",
             "HMSMTNCONSQJOTKOPAOMQYNDPUQWUQJUEGNWAYGDLIT",
             "HQBKSKIXQMKEOVVMAHXLBOQQJXEYMBUMBOEFUDBAPWU",
             "HQDHFCNDPFUXOIXKPUMIQJJFOKCYELPTJPBSPOFGAPL",
             "HSARYDPNQIYYBGSXBFOFLRRPSWDEQGOSMSCONFEBVRP",
             "HVHDXLMSMKIBUMCNKOCPSPJMTFNPEDJQLNFOBTTHMPT",
             "HVNTBSGWPIJIQFTJZIGRJTDXWLMDPWSVUNEFVSBMYFE"]

plot(Fig3_18ssa, prefix="Numerical_experiments_fig08", save_path='./')
plot(Fig3_18ssb, prefix="Numerical_experiments_fig09", save_path='./')
plot(Fig3_18ssc, prefix="Numerical_experiments_fig10", save_path='./')
print("fin")  # remove or comment to display in jupyter

In [ ]:
Fig3_64ss = ["HDCEEYSMMMMMMMMMMWMMMMNCMMMMMMMMMMMMMMMMMMM",
             "HFOWFIPMMMMMMMMMMWMMMMRCMMMMMMMMMMMMMMMMMMM",
             "HIOMYNHMMMMMMMMMMCMMMMFWMMMMMMMMMMMMMMMMMMM",
             "HROXTCGMMMMMMMMMMWMMMMCCMMMMMMMMMMMMMMMMMMM",
             "HVEMBUMMMMMMMMMMMCMMMMNWMMMMMMMMMMMMMMMMMMM",
            ]
#plot(chapter3)i
plot(Fig3_64ss, prefix="Numerical_experiments_fig11", save_path='./')
print("fin")

In [ ]:
def tr(i): return round((i - 32) * 9.5 / 95 - 4.5, 1)
for l in "EAGHNFODVNJCP":
    print(tr(ord(l)))

In [ ]:
param_str = "EAGHNFODVNJCP"
npoints = 100000
np.savetxt( "EAG.txt", prog08aa(param_str, npoints))